### Realizado por: 
#### Desiderio Nevado López
#### Laura Martín Olinero 
#### Daniel Quesada Pérez de San Roman

# Apartado 2 Consulta 2

### Calcular la media de concentración de humo en la estancia utilizando ventanas deslizantes de 15 segundos de duración cada una, con un valor de deslizamiento de 9 segundos entre ventanas consecutivas.

#### Pd: Esta consulta la he realizado de dos formas diferentes que pueden ser válidas

## Forma 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, window, col, mean, stddev
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, LongType, DoubleType
import time

# Iniciamos sesion de Spark
spark = SparkSession.builder.master('local[*]').appName('Consulta2_Apartado2').getOrCreate()

# Establecemos un nuevo nivel de log para evitar warning al hacer login
spark.sparkContext.setLogLevel('error')

in_path = 'data/output/split_files/spark'
out_path = 'data/output/4_3'
kafka_host = 'localhost'
kafka_port = 9092
kafka_topic = 'iot'

#Definimos el esquema de los datos
schema = StructType([
    StructField('ts', TimestampType()),
    StructField('device', StringType()),
    StructField('co', DoubleType()),
    StructField('humidity', DoubleType()),
    StructField('light', BooleanType()),
    StructField('lpg', DoubleType()),
    StructField('motion', BooleanType()),
    StructField('smoke', DoubleType()),
    StructField('temp', DoubleType()),
    StructField('file_name', StringType()),
    StructField('event_time', LongType()),
    ])

In [ ]:
#Lectura de stream de datos (OPCIÓN 2)
df = spark.readStream.format('kafka') \
    .option('kafka.bootstrap.servers', f'{kafka_host}:{kafka_port}') \
    .option('subscribe', kafka_topic) \
    .load() \
    .select(from_json(col('value').cast(StringType()), schema).alias('struct')) \
    .select('struct.*') \
    .withColumn('event_time', col('event_time').cast(TimestampType())) \
    .withColumn('smoke', col('smoke').cast(DoubleType())) \
    .filter('smoke no es nulo')

In [ ]:
# Consulta 2 Apartado 2

avg_smoke = df.withWatermark('event_time', '15 seconds') \
    .groupBy(window('event_time', '15 seconds', '9 seconds')) \
    .agg(avg('smoke').alias('avg_smoke')) \
    .select('window.*', 'avg_smoke')

In [ ]:
avg_smoke.writeStream \
    .outputMode('append') \
    .format('csv') \
    .option('header', True) \
    .option('path', out_path) \
    .start() \
    .awaitTermination()

## Forma 2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, window, col, mean, stddev
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, LongType, DoubleType
import time

# Iniciamos sesion de Spark
spark = SparkSession.builder.master('local[*]').appName('Consulta2_Apartado2').getOrCreate()

# Establecemos un nuevo nivel de log para evitar warning al hacer login
spark.sparkContext.setLogLevel('error')

in_path = 'data/output/split_files/spark'
out_path = 'data/output/4_3'
kafka_host = 'localhost'
kafka_port = 9092
kafka_topic = 'iot'

In [ ]:
# Para leer los datos

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "iot") \
    .load() \
    .selectExpr("CAST(value AS STRING)")

In [ ]:
from pyspark.sql.funticons import split, expr, col

Stream = stream.withColumn('value', expr("regexp_replace(value, '\"','')"))

df = (
    Stream
    .select(
        split(col('value'), ',')[0].cast('float').alias('ts'),
        split(col('value'), ',')[1].alias('device'),
        split(col('value'), ',')[2].cast('float').alias('co'),
        split(col('value'), ',')[3].cast('float').alias('humidity'),
        split(col('value'), ',')[4].cast('boolean').alias('light'),
        split(col('value'), ',')[5].cast('float').alias('lpg'),
        split(col('value'), ',')[6].cast('boolean').alias('motion'),
        split(col('value'), ',')[7].cast('float').alias('smoke'),
        split(col('value'), ',')[8].cast('float').alias('temp'),
    )
)

In [ ]:
# Consulta 2 Apartado 2

avg_smoke = df.withWatermark('event_time', '15 seconds') \
    .groupBy(window('event_time', '15 seconds', '9 seconds')) \
    .agg(avg('smoke').alias('avg_smoke')) \
    .select('window.*', 'avg_smoke')

In [ ]:
avg_smoke.writeStream \
    .outputMode('append') \
    .format('csv') \
    .option('header', True) \
    .option('path', out_path) \
    .start() \
    .awaitTermination()